In [14]:
import chromadb
import pandas as pd

In [15]:
client = chromadb.Client()
collection = client.get_collection("test_collection")
# collection = client.create_collection("test_collection")

In [11]:
collection.add(
    documents=["このページの本文へ"],
    ids=["doc3"],
)

Add of existing embedding ID: doc3
Insert of existing embedding ID: doc3


In [16]:
results = collection.query(
    query_texts=["This is document2"],
    n_results=3
)
results

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


{'ids': [['doc2', 'doc1', 'doc3']],
 'distances': [[0.0, 0.2221541851758957, 1.6004157066345215]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['This is document2', 'This is document1', 'このページの本文へ']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [18]:
df = pd.DataFrame({
    'ids': results['ids'][0],
    'distances': results['distances'][0],
    'metadatas': results['metadatas'][0],
    'documents': results['documents'][0]
})
df

,ids,distances,metadatas,documents
0,doc2,0.000000,None,This is document2
1,doc1,0.222154,None,This is document1
2,doc3,1.600416,None,このページの本文へ


In [22]:
import re

def smart_chunk_text(text, max_length):
    sentences = re.split(r'(?<=[。！？])', text)
    sentences = [sentence for sentence in sentences if sentence.strip()]
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) > max_length:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += sentence
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

# Пример текста и максимальная длина чанков
text = """2016年12月16日から18日の日程で、韓国ギョンギ道のギョンギジムにおいて開催された、世界最大規模のeスポーツ世界大会「Intel Extreme Masters（IEM）」ギョンギ大会。最終日の18日に、今回の大会で競われた3タイトルの決勝戦が行われた。
今回のIEMギョンギ大会で競われた種目は「Overwatch」、「StarCraft II」、そして「League of Legends」の3タイトル。 こちらの記事 で紹介しているように、16日と17日にそれぞれの種目でトーナメント戦を開催して決勝進出2チームを選出。そして、最終日の18日に全種目の決勝戦が行われた。
まず最初に決勝戦が行われたのが、Overwatchだ。6人対6人のチーム戦で、最大5戦の3戦先勝マッチで競った。決勝に進出したのは、チーム「Lunatic Hai」と、チーム「Luxury Watch Red」。どちらも準々決勝を3勝0敗、準決勝を3勝1敗と、圧倒的な強さで勝ち上がってきたチームで、決勝戦は実力伯仲。実際、決勝戦の最初の2戦は、双方が1勝ずつと、期待通りの立ち上がりとなった。しかし、その後Luxury Watch Redチームが2連勝を成し遂げ、3勝1敗で優勝となった。
続いて開催されたのが、StarCraft IIの決勝戦。こちらは1対1の個人戦種目で、最大7戦の4戦先勝マッチで行われた。決勝に勝ち進んだのは、予選から全戦無敗で勝ち上がってきた「INnoVation」選手と、予選AブロックでINnoVation選手に1敗したものの、それ以降全勝して勝ち上がった「Stats」選手の2名。予選での戦いぶりから、INnoVation選手が頭一つ抜け出た実力があると感じていたが、決勝戦でもINnoVation選手が実力を遺憾なく発揮し、4勝0敗と一方的な試合内容で完勝。予選から決勝まで、一度も負けることなく優勝を勝ち取った。
最後の決勝戦となったのがLeague of Legends。こちらは5人対5人のチーム戦で、最大5戦の3戦先勝マッチで競った。決勝に出場したのは、予選から無敗で勝ち上がってきたチーム「Samsung Galaxy」と、プレーオフで1度だけ負けたが、危なげなく勝ち上がってきたチーム「Kongdoo Master」の2チームだ。どちらも韓国内の人気チームで、会場には双方のサポーターが詰めかけて応援にも力が入っており、試合内容も同様に手に汗握るものとなった。1試合のプレー時間が40分ほどに及ぶ長丁場となる中、まずSamsung Galaxyチームが2戦先勝して優勝にリーチをかけた。追い詰められたKongdoo Masterチームだったが、ここで気合いを入れ直し3戦目で勝利をもぎ取る。しかし、4戦目はまたもやSamsung Galaxyチームにリードを許し、結局3勝1敗でSamsung Galaxyチームの優勝で幕を閉じた。
今回筆者は、IEM大会を初めて取材したが、その規模の大きさに驚かされた。世界的にもeスポーツが盛んな国として認知されている韓国だからこそ、プロのバスケットボールチームがホームグラウンドにするような大きな体育館を会場にできたのだろう。まだまだeスポーツが認知されておらず、小規模な大会を開催することすら難しい状況の日本とは、大きな隔たりがあると感じた。
また、韓国でのeスポーツの人気の高さも伝わってきた。観戦するには有料チケットが必要だが、それでも大勢の観客が集まっていたし、人気プレーヤーの追っかけも多く見かけた。そして、各種目の優勝チームと準優勝チームのサイン会がロビーで行われていたが、若い女性を中心とした長蛇の列ができていた点からも、人気の高さを如実に感じれた。
今回の大会に参加したチームは、そのほとんどが韓国チームだった点も驚いた部分だ。もちろん、地元韓国での開催という部分もあるが、IEMは大規模な国際大会で、地元以外の国からのエントリーも少なくなく、当然本戦に出場するには予選を勝ち抜く必要がある。そう考えると、韓国eスポーツ界の層の厚さと実力の高さについて、疑いの余地はないだろう。
ただ、それだけeスポーツの人気も実力もある韓国で開催した大会ながら、会場にはかなり多くの空席が目立っていたのは気になった。予選の2日間は座席が1割ほどしか埋まらず、決勝戦が行われた最終日でもかろうじてアリーナ席が7割近く埋まった程度で、その他の座席はほぼ空席。全座席に対して4割に満たないほどの観客数だった。ソウルから30kmほど離れた場所でややアクセスが不便だったことや、韓国のeスポーツ専門チャンネルが連日試合を生中継していたことも影響しているかもしれないが、それでも会場の観客の少なさはかなり気になった。決勝戦は十分な盛り上がりを見せたが、大会運営には課題が残されたと感じた。今回の大会内容を分析し、今後のよりよい大会運営に活かされることを期待したい。
今回、IEMギョンギ大会に参加して、やはり日本でもeスポーツがもっと盛り上がってほしいという気持ちが一層強くなった。以前に比べて、日本でもeスポーツの認知度は高まりつつあるが、まだ世界的な舞台に立つまでに至っていないと再認識させられた。eスポーツを他のスポーツと同様に扱うだけでなく、"eスポーツ＝とはいえ単なるゲーム"という日本独特の意識も変革しながら、競技人口を増やし、実力を高めることが欠かせないだろう。その上で、官民一体となって盛り上げてもらいたいと思う。"""
max_length = 500

# Чанкинг текста
chunks = smart_chunk_text(text, max_length)
for chunk in chunks:
    print(chunk)
    print()


2016年12月16日から18日の日程で、韓国ギョンギ道のギョンギジムにおいて開催された、世界最大規模のeスポーツ世界大会「Intel Extreme Masters（IEM）」ギョンギ大会。最終日の18日に、今回の大会で競われた3タイトルの決勝戦が行われた。
今回のIEMギョンギ大会で競われた種目は「Overwatch」、「StarCraft II」、そして「League of Legends」の3タイトル。 こちらの記事 で紹介しているように、16日と17日にそれぞれの種目でトーナメント戦を開催して決勝進出2チームを選出。そして、最終日の18日に全種目の決勝戦が行われた。
まず最初に決勝戦が行われたのが、Overwatchだ。6人対6人のチーム戦で、最大5戦の3戦先勝マッチで競った。決勝に進出したのは、チーム「Lunatic Hai」と、チーム「Luxury Watch Red」。どちらも準々決勝を3勝0敗、準決勝を3勝1敗と、圧倒的な強さで勝ち上がってきたチームで、決勝戦は実力伯仲。実際、決勝戦の最初の2戦は、双方が1勝ずつと、期待通りの立ち上がりとなった。

しかし、その後Luxury Watch Redチームが2連勝を成し遂げ、3勝1敗で優勝となった。
続いて開催されたのが、StarCraft IIの決勝戦。こちらは1対1の個人戦種目で、最大7戦の4戦先勝マッチで行われた。決勝に勝ち進んだのは、予選から全戦無敗で勝ち上がってきた「INnoVation」選手と、予選AブロックでINnoVation選手に1敗したものの、それ以降全勝して勝ち上がった「Stats」選手の2名。予選での戦いぶりから、INnoVation選手が頭一つ抜け出た実力があると感じていたが、決勝戦でもINnoVation選手が実力を遺憾なく発揮し、4勝0敗と一方的な試合内容で完勝。予選から決勝まで、一度も負けることなく優勝を勝ち取った。
最後の決勝戦となったのがLeague of Legends。こちらは5人対5人のチーム戦で、最大5戦の3戦先勝マッチで競った。決勝に出場したのは、予選から無敗で勝ち上がってきたチーム「Samsung Galaxy」と、プレーオフで1度だけ負けたが、危なげなく勝ち上がってきたチーム「Kongdoo Master」の2チームだ。

どちらも韓国内の人気チームで、